In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [3]:
file_path = "vs_method.xlsx"
vs = pd.read_excel(file_path, sheet_name=None)

# Preprocess data
vs_dct = {}
for sheet_name, df in vs.items():
    df['Ticker'] = pd.to_datetime(df['Ticker']) + pd.offsets.BMonthEnd(1)
    vs_dct[sheet_name] = df.ffill().fillna(0)

# Separate target and features
y = vs_dct['2024 Return']
del vs_dct['2024 Return']
x = vs_dct
rebal_dt = x['Active EQ MF_Ret']['Ticker']

In [6]:
# Build regression input dictionaries
reg_x = {}
reg_y = {}
for row in range(len(rebal_dt)):
    reg_x[row] = {k: x[k].iloc[row] for k in x}
    reg_y[row] = {k: y[k].iloc[row] for k in y}

# Collect standardized beta coefficients, R2, and standard errors
results = []

for rebal in range(len(rebal_dt)):
    # Combine feature and target data into one DataFrame
    reg_df = pd.concat([pd.DataFrame(reg_x[rebal]), pd.Series(reg_y[rebal])], axis=1)
    reg_df.columns = list(reg_df.columns[:-1]) + ['Target']
    reg_df = reg_df.dropna()
    reg_df = reg_df.apply(pd.to_numeric, errors='coerce').dropna()

    if reg_df.shape[0] <= 1:
        continue

    X = reg_df.drop(columns='Target')
    y_target = reg_df['Target']

    # Standardize features and target
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y_target.values.reshape(-1, 1)).ravel()

    # Fit sklearn linear regression for R2
    sk_model = LinearRegression().fit(X_scaled, y_scaled)
    r2 = sk_model.score(X_scaled, y_scaled)

    # Fit statsmodels OLS to extract standard errors
    X_const = sm.add_constant(X_scaled)
    sm_model = sm.OLS(y_scaled, X_const).fit()
    se = dict(zip(['const'] + list(X.columns), sm_model.bse))

    # Compile results
    result = dict(zip(X.columns, sk_model.coef_))
    result.update({f"{k}_SE": v for k, v in se.items() if k != 'const'})
    result['R2'] = r2
    result['RebalDate'] = rebal_dt[rebal]
    results.append(result)

# Convert to DataFrame
final_df = pd.DataFrame(results)
final_df

,Active EQ MF_Ret,Active EQ MF_IR,Active EQ MF_SR,Active EQ MF_TE,Active EQ MF_Exp,Active EQ MF_PEG,Active EQ MF_Medalist,Active EQ MF_Ret_SE,Active EQ MF_IR_SE,Active EQ MF_SR_SE,Active EQ MF_TE_SE,Active EQ MF_Exp_SE,Active EQ MF_PEG_SE,Active EQ MF_Medalist_SE,R2,RebalDate
0,-1.364351,0.208563,1.087409,-0.006126,0.156996,0.018015,0.262829,0.959321,0.233659,0.839105,0.334198,0.233275,0.235071,0.234896,0.255327,2019-01-31
1,-0.386532,0.275391,0.513195,0.002472,0.138857,0.270248,0.224136,0.893250,0.229214,0.766179,0.346551,0.209995,0.209965,0.209050,0.369124,2019-02-28
2,1.422075,0.119661,-0.892765,-0.740349,0.260556,0.124465,0.220702,0.953051,0.213023,0.860524,0.330657,0.179449,0.178252,0.183833,0.540126,2019-03-29
3,1.982862,0.326328,-1.471668,-0.914982,0.144175,0.180774,0.005825,0.662600,0.160329,0.610311,0.220935,0.137517,0.133995,0.134949,0.754681,2019-04-30
4,2.762822,0.404399,-2.540749,-1.139051,0.288857,0.071053,0.166122,0.759570,0.187159,0.747533,0.230354,0.154340,0.162453,0.150745,0.665576,2019-05-31
5,2.043114,0.518848,-1.892896,-0.744458,0.183821,0.010424,0.041503,0.590950,0.179258,0.578840,0.196439,0.149407,0.150512,0.147704,0.674048,2019-06-28
6,2.323038,0.610510,-2.281700,-0.625171,0.125162,0.035400,0.086119,0.645765,0.191963,0.669020,0.198459,0.149258,0.146941,0.147504,0.662866,2019-07-31
7,2.720200,0.663290,-2.802878,-0.586003,0.098138,-0.102007,0.098287,0.720198,0.213553,0.803204,0.187019,0.148135,0.156440,0.153036,0.649617,2019-08-30
8,2.764046,0.691756,-3.002779,-0.649689,0.329812,-0.247200,0.165593,0.658872,0.198423,0.754555,0.187066,0.153734,0.165126,0.151121,0.672182,2019-09-30
9,1.418954,0.615419,-1.530510,-0.375557,0.281413,-0.252124,-0.027002,0.485537,0.227710,0.554082,0.177668,0.166933,0.192628,0.189370,0.588284,2019-10-31


In [9]:
import plotly.express as px
import plotly.graph_objects as go

# Step 1: Normalize absolute standardized beta coefficients to sum to 100%
beta_columns = [col for col in final_df.columns if col in [
    'Active EQ MF_Ret', 'Active EQ MF_IR', 'Active EQ MF_SR', 'Active EQ MF_TE',
    'Active EQ MF_Exp', 'Active EQ MF_PEG', 'Active EQ MF_Medalist'
]]

normalized_df = final_df.copy()
normalized_df[beta_columns] = normalized_df[beta_columns].abs()
normalized_df[beta_columns] = normalized_df[beta_columns].div(normalized_df[beta_columns].sum(axis=1), axis=0) * 100

# Step 2: Calculate 12-month trailing average
normalized_df_sorted = normalized_df.sort_values("RebalDate").reset_index(drop=True)
rolling_avg_df = normalized_df_sorted[["RebalDate"] + beta_columns].copy()

for col in beta_columns:
    rolling_avg_df[col] = rolling_avg_df[col].rolling(window=12, min_periods=1).mean()

# Step 3: Plot area chart of rolling 12-month average beta weights
fig_area = go.Figure()
for col in beta_columns:
    fig_area.add_trace(go.Scatter(
        x=rolling_avg_df['RebalDate'],
        y=rolling_avg_df[col],
        stackgroup='one',
        name=col
    ))
fig_area.update_layout(title="12-Month Rolling Average of Feature Weights (Normalized Betas)",
                       xaxis_title="Rebalance Date", yaxis_title="Weight (%)")
fig_area.show()

# Step 4: Plot R² as a line chart
fig_r2 = px.line(normalized_df, x="RebalDate", y="R2", title="R² Over Time")
fig_r2.update_traces(mode="lines+markers")
fig_r2.update_layout(yaxis_title="R-squared")
fig_r2.show()

# Step 5: Plot average standard error over time
se_columns = [col for col in final_df.columns if col.endswith("_SE")]
normalized_df["SE_Avg"] = normalized_df[se_columns].mean(axis=1)
fig_se = px.line(normalized_df, x="RebalDate", y="SE_Avg", title="Average Standard Error Over Time")
fig_se.update_traces(mode="lines+markers")
fig_se.update_layout(yaxis_title="Average SE")
fig_se.show()

# Step 6: Compute t-values and p-values
t_p_results = []

for rebal in range(len(rebal_dt)):
    reg_df = pd.concat([pd.DataFrame(reg_x[rebal]), pd.Series(reg_y[rebal])], axis=1)
    reg_df.columns = list(reg_df.columns[:-1]) + ['Target']
    reg_df = reg_df.dropna()
    reg_df = reg_df.apply(pd.to_numeric, errors='coerce').dropna()

    if reg_df.shape[0] <= 1:
        continue

    X = reg_df.drop(columns='Target')
    y_target = reg_df['Target']

    X_scaled = StandardScaler().fit_transform(X)
    y_scaled = StandardScaler().fit_transform(y_target.values.reshape(-1, 1)).ravel()

    X_const = sm.add_constant(X_scaled)
    model = sm.OLS(y_scaled, X_const).fit()

    feature_names = ['const'] + list(X.columns)
    tvals = dict(zip(feature_names, model.tvalues))
    pvals = dict(zip(feature_names, model.pvalues))

    summary_data = {f"{k}_tval": t for k, t in tvals.items() if k != 'const'}
    summary_data.update({f"{k}_pval": p for k, p in pvals.items() if k != 'const'})
    summary_data['RebalDate'] = rebal_dt[rebal]
    t_p_results.append(summary_data)

tp_df = pd.DataFrame(t_p_results)
tp_df


,Active EQ MF_Ret_tval,Active EQ MF_IR_tval,Active EQ MF_SR_tval,Active EQ MF_TE_tval,Active EQ MF_Exp_tval,Active EQ MF_PEG_tval,Active EQ MF_Medalist_tval,Active EQ MF_Ret_pval,Active EQ MF_IR_pval,Active EQ MF_SR_pval,Active EQ MF_TE_pval,Active EQ MF_Exp_pval,Active EQ MF_PEG_pval,Active EQ MF_Medalist_pval,RebalDate
0,-1.422204,0.892597,1.295915,-0.018329,0.673007,0.076635,1.118918,0.168991,0.381733,0.208437,0.985541,0.507951,0.939607,0.275244,2019-01-31
1,-0.432726,1.201454,0.669811,0.007133,0.661239,1.287111,1.072162,0.669426,0.242350,0.509948,0.994373,0.515325,0.211433,0.295268,2019-02-28
2,1.492128,0.561727,-1.037467,-2.239027,1.451977,0.698255,1.200558,0.149868,0.579980,0.310788,0.035592,0.160621,0.492333,0.242691,2019-03-29
3,2.992549,2.035363,-2.411341,-4.141408,1.048419,1.349110,0.043165,0.006709,0.054036,0.024686,0.000427,0.305828,0.191024,0.965959,2019-04-30
4,3.637352,2.160726,-3.398845,-4.944779,1.871571,0.437378,1.102000,0.001454,0.041873,0.002578,0.000060,0.074623,0.666101,0.282372,2019-05-31
5,3.457340,2.894421,-3.270155,-3.789771,1.230338,0.069256,0.280987,0.002242,0.008412,0.003501,0.001006,0.231565,0.945411,0.781346,2019-06-28
6,3.597345,3.180349,-3.410509,-3.150129,0.838562,0.240913,0.583845,0.001602,0.004328,0.002508,0.004646,0.410734,0.811854,0.565263,2019-07-31
7,3.777017,3.105972,-3.489620,-3.133383,0.662489,-0.652055,0.642250,0.001037,0.005153,0.002075,0.004832,0.514539,0.521120,0.527348,2019-08-30
8,4.195118,3.486263,-3.979538,-3.473055,2.145341,-1.497040,1.095767,0.000375,0.002092,0.000634,0.002159,0.043220,0.148593,0.285032,2019-09-30
9,2.922440,2.702647,-2.762248,-2.113812,1.685781,-1.308864,-0.142590,0.007888,0.013001,0.011368,0.046102,0.105971,0.204088,0.887912,2019-10-31


In [10]:
import plotly.express as px

# Extract standardized beta coefficients and corresponding SEs from final_df
beta_cols = [
    'Active EQ MF_Ret', 'Active EQ MF_IR', 'Active EQ MF_SR',
    'Active EQ MF_TE', 'Active EQ MF_Exp', 'Active EQ MF_PEG',
    'Active EQ MF_Medalist'
]

se_cols = [col + "_SE" for col in beta_cols]

# Melt the dataframe for beta values
beta_melt = final_df.melt(
    id_vars=["RebalDate"], 
    value_vars=beta_cols, 
    var_name="Feature", 
    value_name="Beta"
)

# Melt the dataframe for standard errors
se_melt = final_df.melt(
    id_vars=["RebalDate"], 
    value_vars=se_cols, 
    var_name="Feature_SE", 
    value_name="SE"
)

# Align feature names
se_melt["Feature"] = se_melt["Feature_SE"].str.replace("_SE", "")

# Merge both for plotting
merged_df = pd.merge(beta_melt, se_melt[["RebalDate", "Feature", "SE"]], on=["RebalDate", "Feature"])

# Plot SE vs. Beta magnitude
fig = px.scatter(
    merged_df,
    x="Beta",
    y="SE",
    color="Feature",
    hover_data=["RebalDate"],
    title="Standard Error vs. Beta Coefficient Magnitude",
    labels={"Beta": "Standardized Beta", "SE": "Standard Error"}
)
fig.update_traces(marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')))
fig.show()


In [21]:
import plotly.graph_objects as go

# Prepare long-form DataFrame for standard error line plotting
se_trend_df = final_df[["RebalDate"] + se_cols].copy()
se_trend_long = se_trend_df.melt(id_vars="RebalDate", var_name="Feature", value_name="Standard Error")
se_trend_long["Feature"] = se_trend_long["Feature"].str.replace("_SE", "", regex=False)

# Create line graph using plotly
fig_se_trend = go.Figure()

for feature in se_trend_long["Feature"].unique():
    feature_df = se_trend_long[se_trend_long["Feature"] == feature]
    fig_se_trend.add_trace(go.Scatter(
        x=feature_df["RebalDate"],
        y=feature_df["Standard Error"],
        mode='lines+markers',
        name=feature
    ))

fig_se_trend.update_layout(
    title="Standard Error Trend per Feature Over Time",
    xaxis_title="Rebalance Date",
    yaxis_title="Standard Error",
    legend_title="Feature"
)

fig.update_layout(
    uirevision='keep_axes',  # any non-changing string works
    #xaxis=dict(
     #   range=[xmin, xmax],   # set this explicitly
      #  fixedrange=False      # optional: set True to lock zoom/pan
    #),
    yaxis=dict(
        range=[0, 3.5],   # set this explicitly
        fixedrange=False
    )
)

fig_se_trend.show()


In [14]:
# Compute average p-values and average absolute t-values for each feature
features = ['Active EQ MF_Ret', 'Active EQ MF_IR', 'Active EQ MF_SR',
            'Active EQ MF_TE', 'Active EQ MF_Exp', 'Active EQ MF_PEG',
            'Active EQ MF_Medalist']

# Extract relevant columns
pval_cols = [f"{f}_pval" for f in features]
tval_cols = [f"{f}_tval" for f in features]

# Compute averages
avg_pvals = tp_df[pval_cols].mean().rename(lambda x: x.replace('_pval', ''))
avg_tvals = tp_df[tval_cols].abs().mean().rename(lambda x: x.replace('_tval', ''))

# Combine and rank
ranking_df = pd.DataFrame({
    'Avg_Abs_TValue': avg_tvals,
    'Avg_PValue': avg_pvals
})
ranking_df['Rank_by_TValue'] = ranking_df['Avg_Abs_TValue'].rank(ascending=False)
ranking_df['Rank_by_PValue'] = ranking_df['Avg_PValue'].rank(ascending=True)
ranking_df['CompositeRank'] = (ranking_df['Rank_by_TValue'] + ranking_df['Rank_by_PValue']) / 2
ranking_df = ranking_df.sort_values(by='CompositeRank')
ranking_df
#tools.display_dataframe_to_user(name="Feature Significance Ranking", dataframe=ranking_df)


,Avg_Abs_TValue,Avg_PValue,Rank_by_TValue,Rank_by_PValue,CompositeRank
Active EQ MF_Ret,1.406870,0.414802,1.0,2.0,1.5
Active EQ MF_TE,1.294605,0.386445,2.0,1.0,1.5
Active EQ MF_SR,1.119285,0.501873,3.0,5.0,4.0
Active EQ MF_PEG,0.964061,0.455257,5.0,3.0,4.0
Active EQ MF_IR,0.973792,0.507834,4.0,6.0,5.0
Active EQ MF_Exp,0.767985,0.498334,6.0,4.0,5.0
Active EQ MF_Medalist,0.674370,0.539169,7.0,7.0,7.0
